In [15]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import threading

In [16]:
cam = 0
fps = 240 #Maximum achivable was 163.06

model_version = "my_model_12.0.keras"
max_frames = 40  # Number of frames to evaluate

image_path = "Image0001.jpg"
frame_buffer = []

Im_Width = 128
Im_Height = 128

#crop
x = 220
y = 0
w = 200
h = 200

max_brightness = 100
#max_edgeRichness = 

run = True

In [17]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [18]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [19]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [20]:
def select_best_frame(frame_buffer, x, y, w, h, max_brightness):

    best_frame = None
    highest_brightness = 0
    highest_edge_richness = 0

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, x, y, w, h)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > max_brightness :
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
        
        elif brightness > 50 and brightness < 100:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame


    return best_frame  # Returns None if no frames meet the threshold


In [21]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [22]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        return "Non-Defective", prediction       
        
    else:
        return "Defective", prediction

In [23]:
import cv2

def capture(cam, fps, frame_buffer, max_frames):
    
    cap = cv2.VideoCapture(cam)  # Open webcam
    cap.set(cv2.CAP_PROP_FPS, fps)  # Set the desired FPS

    while run:
        if not cap.isOpened():
            print("Error: Cannot open camera.")
            return

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            frame_buffer.append(frame)

            # Yield the buffer when it's full
            if len(frame_buffer) == max_frames:
                yield frame_buffer  # Yield the full buffer
                print(":)")
                print("\n")
                frame_buffer = []  # Clear the buffer for the next batch

    cap.release()

In [24]:
# Thread target to consume the generator
def run_generator(generator):
    for buffer in generator:
        print(f"Buffer received with {len(buffer)} frames.")

In [25]:
frame_generator = capture(cam, fps, frame_buffer, max_frames)

In [26]:
def runModel(model_version, frame_buffer, x, y, w, h, max_brightness, target_size=(Im_Width, Im_Height)):

    # Load the model
    print("Loading model...") 
    model = load_model(model_version)
    print("Model loaded successfully.")
    
    while True:
        best_frame = select_best_frame(frame_buffer, x, y, w, h, max_brightness)
        # Process and predict
        if best_frame is not None:
            feedback, confidence = predict_frame(best_frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            prob = abs(round(confidence,2)- 0.5)*200

            # Display feedback on the frame
            cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(best_frame, f"Confidence: {prob:.0f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Display the best frame
            cv2.imshow("Best Frame", best_frame)

        else:
            # Load and display the "no-bottle" image in the same window
            image = cv2.imread(image_path)
            cv2.putText(image, "No Bottle!", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Best Frame", image)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            run = False
            break

    cv2.destroyAllWindows()
    return run

In [29]:
#capture_thread = threading.Thread(target=capture, args=(cam, fps, frame_buffer, max_frames))
generator_thread = threading.Thread(target=run_generator, args=(frame_generator,))
runModel_thread = threading.Thread(target=runModel, args=(model_version, frame_buffer, x, y, w, h, max_brightness))

In [30]:
if __name__ == "__main__":

    # Start the threads
    generator_thread.start()
    #capture_thread.start()
    runModel_thread.start()

    # Wait for threads to complete
    #generator_thread.join()
    #capture_thread.join()
    #runModel_thread.join()


Loading model...


Model loaded successfully.
